## Sentiment Analizi & Sınıflandırma Problemleri

In [64]:
from textblob import TextBlob
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import pandas as pd
data = pd.read_csv('train.tsv',sep='\t')

In [3]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


#### Yapacaklarımız

Verisetinde hazır sentiment değişkeni oluşturulmuş. Biz bu değişken olmasaydı bile kendimiz bunu oluşturacaktık. <br>

* Sentiment değişkenini oluşturma

Hazır oluşturulmuş sentiment  0-4 arasında değer almakta... <br><br>

0-1 : kötü yorum skorları<br>
2 : orta sınıf <br>
3-4 : iyi yorum skoraları<br><br>

* biz bu aralığı negative-positive taglemesi yapacağız....<br>

#### NEG-POS Taglemesi

In [4]:
# negatifler

In [5]:
data['Sentiment'].replace(0,value = 'negative',inplace=True)
data['Sentiment'].replace(1,value = 'negative',inplace=True)

In [6]:
# pozitifler

In [7]:
data['Sentiment'].replace(3,value = 'positive',inplace=True)
data['Sentiment'].replace(4,value = 'positive',inplace=True)

In [8]:
# 2 değerini dışarıda bırakalım.....

In [9]:
data = data[data.Sentiment != 2]

In [10]:
data.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,negative
21,22,1,good for the goose,positive
22,23,1,good,positive
33,34,1,"the gander , some of which occasionally amuses...",negative
46,47,1,amuses,positive


In [11]:
# kontrol edelim

In [12]:
data.groupby('Sentiment').count()

,PhraseId,SentenceId,Phrase
Sentiment,,,
negative,34345,34345,34345
positive,42133,42133,42133


In [13]:
# metini ön işlemeden önce dataframe i 2 değişkene indirgeyelim...

In [14]:
df = pd.DataFrame()
df['text'] = data['Phrase'].copy()
df['label'] = data['Sentiment'].copy()

In [15]:
df.head()

,text,label
0,A series of escapades demonstrating the adage ...,negative
21,good for the goose,positive
22,good,positive
33,"the gander , some of which occasionally amuses...",negative
46,amuses,positive


#### Metin Ön İşleme

In [16]:
# lower
df['text'] = df['text'].apply(lambda x : x.lower())

# noktalama işaretleri
df['text'] = df['text'].str.replace('[^\w\s]','')

# sayılar
df['text'] = df['text'].str.replace('\d','')

# stopwords
import nltk

from nltk.corpus import stopwords
sw = stopwords.words('english')

df['text'] = df['text'].apply(lambda x : ' '.join(word for word in x.split() if word not in sw))

# seyreklerinin silinmesi
sprase_word = pd.Series(' '.join(df['text']).split()).value_counts()[-1000:]

df['text'] = df['text'].apply(lambda x : ' '.join(word for word in x.split() if word not in sprase_word))

# lemmi

from textblob import Word
df['text'] = df['text'].apply(lambda x : ' '.join(Word(w).lemmatize() for w in x.split()))

In [17]:
df.head()

,text,label
0,series demonstrating adage good goose also goo...,negative
21,good goose,positive
22,good,positive
33,gander occasionally amuses none amount much story,negative
46,amuses,positive


## Değişken Mühendisliği (Feature Engineering)
Amaç bir texti makine öğrenimine sokabilmek için gereken sayısal verileri üretmektir. <br>

* Count Vectors (bütün gözlemlerdeki unique kelimeler bir değişken olarak atanır ve gözlemlerde görülme sıklığı incelenir.)
* TF-IDF Vectors (***words ,characters, n-grams***)
* Word Embeddings (bütün kelimelerden bir kelime uzayı çıkartılıp gözlemin kelime yoğunluğu hesaplanır.  [wiki](https://en.wikipedia.org/wiki/Word_embedding))

TF(t) = (Bir t teriminin bir dökümanda gözlenme frekansı) / (dökümandaki toplam terim sayısı) <br>

IDF(t) = log_e(Toplam döküman sayısı / içeride t terimi olan belge sayısı)<br>

### Test-Train

In [18]:
train_x, test_x , train_y ,test_y = model_selection.train_test_split(df['text'],df['label'])

In [19]:
train_x[:2]

63735                        oldhat province male intrigue
3962     feel like light errol morris focusing eccentri...
Name: text, dtype: object

In [20]:
# encoder ile y değerlerini 0-1 olarak atama

In [21]:
encoder = preprocessing.LabelEncoder()

In [22]:
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [23]:
train_y[:10]

array([0, 0, 1, 1, 1, 0, 0, 1, 1, 1])

In [24]:
test_y[:10]

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1])

### Count Vectors

In [25]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

CountVectorizer()

In [26]:
# count yöntemiyle oluşturulmuş train_x

In [27]:
train_x_count = vectorizer.transform(train_x)
test_x_count = vectorizer.transform(test_x)

In [28]:
# sonucu görmek için 

her bir unique kelime bir değişken olarak atandığından get_feature_names bize bu ***değişkenlerin adlarını*** vericek...

In [29]:
vectorizer.get_feature_names()[:5]

['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned']

bu unique değerlerin gözlemde olup olmama değerini (0-1) gösteren matrix

In [30]:
train_x_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### TF-IDF

In [31]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

TfidfVectorizer()

#### word level TF-IDF

In [32]:
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

In [33]:
# future names

In [34]:
tf_idf_word_vectorizer.get_feature_names()[:5]

['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned']

In [35]:
# matrix

In [36]:
x_train_tf_idf_word.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### N-Gram Level TF-IDF
TF-IDF algoritmasını N-gram kullanarak oluşturma

In [37]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2,3))
tf_idf_ngram_vectorizer.fit(train_x)

TfidfVectorizer(ngram_range=(2, 3))

In [38]:
x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(train_x)
x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

In [39]:
# future names

In [40]:
tf_idf_word_vectorizer.get_feature_names()[:5]

['aaa', 'aaliyah', 'abagnale', 'abandon', 'abandoned']

In [41]:
# matrix

In [42]:
x_train_tf_idf_word.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

#### Character Level TF-IDF

In [45]:
tf_idf_char_vectorizer = TfidfVectorizer(analyzer='char',ngram_range=(2,3))
tf_idf_char_vectorizer.fit(train_x)

TfidfVectorizer(analyzer='char', ngram_range=(2, 3))

In [46]:
x_train_tf_idf_char = tf_idf_char_vectorizer.transform(train_x)
x_test_tf_idf_char = tf_idf_char_vectorizer.transform(test_x)

In [47]:
# future names

In [48]:
tf_idf_char_vectorizer.get_feature_names()[:5]

[' a', ' aa', ' ab', ' ac', ' ad']

In [49]:
# matrix

In [50]:
x_train_tf_idf_char.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.06399696, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Makine Öğrenmesi ile Sentiment Sınıfladırması
Farklı Classification Modelleri ile denemeler yapacağız.

### Lojistik Regresyon

#### Count-Vectors

In [95]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(train_x_count,train_y)

accuracy = model_selection.cross_val_score(loj_model,
                                           test_x_count,
                                           test_y,
                                           cv=10).mean()

print('Count Vectors Doğruluk Oranı : ',accuracy)

Count Vectors Doğruluk Oranı :  0.8336820083682008


#### Word-Level TF-IDF

In [52]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_word,train_y)

accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_word,
                                           test_y,
                                           cv=10).mean()

print('Count Vectors Doğruluk Oranı : ',accuracy)

Count Vectors Doğruluk Oranı :  0.8287656903765692


#### N-Gram TF-IDF

In [53]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_ngram,train_y)

accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_ngram,
                                           test_y,
                                           cv=10).mean()

print('Count Vectors Doğruluk Oranı : ',accuracy)

Count Vectors Doğruluk Oranı :  0.747071129707113


#### Char-Level TF-IDF

In [54]:
loj = linear_model.LogisticRegression()
loj_model = loj.fit(x_train_tf_idf_char,train_y)

accuracy = model_selection.cross_val_score(loj_model,
                                           x_test_tf_idf_char,
                                           test_y,
                                           cv=10).mean()

print('Count Vectors Doğruluk Oranı : ',accuracy)

Count Vectors Doğruluk Oranı :  0.7792364016736402


### Naive Bayes

#### Count Vectors

In [55]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(train_x_count,train_y)

accuracy = model_selection.cross_val_score(nb_model,
                                          test_x_count,
                                          test_y,
                                          cv = 10).mean()

print('Count Vectors Doğruluk Oranı : ',accuracy)

Count Vectors Doğruluk Oranı :  0.8312761506276152


#### Word-Level TF-IDF Dogruluk Oranı

In [56]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_word,train_y)

accuracy = model_selection.cross_val_score(nb_model,
                                          x_test_tf_idf_word,
                                          test_y,
                                          cv = 10).mean()

print('Word-Level TF-IDF Doğruluk Oranı : ',accuracy)

Word-Level TF-IDF Doğruluk Oranı :  0.8330020920502091


#### N-Gram TF-IDF

In [57]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_ngram,train_y)

accuracy = model_selection.cross_val_score(nb_model,
                                          x_test_tf_idf_ngram,
                                          test_y,
                                          cv = 10).mean()

print('N-Gram TD-IDF Doğruluk Oranı : ',accuracy)

N-Gram TD-IDF Doğruluk Oranı :  0.7722803347280335


#### Char-Level TF-IDF

In [ ]:
nb = naive_bayes.MultinomialNB()
nb_model = nb.fit(x_train_tf_idf_char,train_y)

accuracy = model_selection.cross_val_score(nb_model,
                                          x_test_tf_idf_ngram_char,
                                          test_y,
                                          cv = 10).mean()

print('Char-Level TF-IDF Doğruluk Oranı : ',accuracy)

### Random Forests

#### Count Vectors

In [86]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(train_x_count,train_y)

accuracy = model_selection.cross_val_score(rf_model,
                                          test_x_count,
                                          test_y,
                                          cv = 10).mean()

print('Count Vectors Doğruluk Oranı',accuracy)

Count Vectors Doğruluk Oranı 0.8240585774058576


#### Word-Level TF-IDF

In [60]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_word,train_y)

accuracy = model_selection.cross_val_score(rf_model,
                                          x_test_tf_idf_word,
                                          test_y,
                                          cv = 10).mean()

print('Count Vectors Doğruluk Oranı',accuracy)

#### N-Gram TF-IDF

In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_ngram,train_y)

accuracy = model_selection.cross_val_score(rf_model,
                                          x_test_tf_idf_ngram,
                                          test_y,
                                          cv = 10).mean()

print('Count Vectors Doğruluk Oranı',accuracy)

#### Char-Level TF-IDF

In [ ]:
rf = ensemble.RandomForestClassifier()
rf_model = rf.fit(x_train_tf_idf_char,train_y)

accuracy = model_selection.cross_val_score(rf_model,
                                          x_test_tf_idf_char,
                                          test_y,
                                          cv = 10).mean()

print('Count Vectors Doğruluk Oranı',accuracy)

### XGBoost 

#### Count Vectors

In [65]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(train_x_count,train_y)

accuracy = model_selection.cross_val_score(xgb_model,
                                          test_x_count,
                                          test_y,
                                          cv = 10).mean()

print('Count Vectors Doğruluk Oranı',accuracy)

[21:56:58] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:56:59] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:57:00] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:57:01] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

#### Word-Level TF-IDF

In [69]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_word,train_y)

accuracy = model_selection.cross_val_score(xgb_model,
                                          x_test_tf_idf_word,
                                          test_y,
                                          cv = 10).mean()

print('Word Level Doğruluk Oranı',accuracy)

[21:59:35] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:59:39] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:59:41] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:59:43] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

#### N-Gram TF-IDF

In [72]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_ngram,train_y)

accuracy = model_selection.cross_val_score(xgb_model,
                                          x_test_tf_idf_ngram,
                                          test_y,
                                          cv = 10).mean()

print('N-Gram Doğruluk Oranı',accuracy)

[22:03:12] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:03:18] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:03:23] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:03:28] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

#### Char-Level TF-IDF

In [73]:
xgb = xgboost.XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_char,train_y)

accuracy = model_selection.cross_val_score(xgb_model,
                                          x_test_tf_idf_char,
                                          test_y,
                                          cv = 10).mean()

print('N-Gram Doğruluk Oranı',accuracy)

[22:13:27] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:13:59] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:14:09] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:14:20] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

In [75]:
# yapılan modellerden lojistik_reg'i seçelim....
# şimdi bunu iş planımıza uygulamamız lazım...
# yani yeni gelen bir yorumun sentiment skorunu çıkarmamız lazım...

### Yeni bir yorumu model nesnemize analiz için sokalım....
Bunun için text değerini ön işlemeye sokmalıyız...

In [105]:
yeni_yorum = pd.Series('This film is very nice and good i like it')

yeni_yorum2 = pd.Series('no, not good look at that shit very bad')

In [106]:
# count vectorizer..

In [107]:
v = CountVectorizer()

In [108]:
# fit etme işlemi train üzerinden olacak...

In [109]:
v.fit(train_x)

CountVectorizer()

In [110]:
# yeni yorumu vektöre transform ettirme

In [111]:
yeni_yorum = v.transform(yeni_yorum)

yeni_yorum2 = v.transform(yeni_yorum2)

In [112]:
# positive algıladı...

In [113]:
loj_model.predict(yeni_yorum)

array([1])

In [114]:
loj_model.predict(yeni_yorum2)

array([0])

In [115]:
# negative algıladı